## Gurobi

Gurobi is a high-performance mathematical optimization solver that supports multiple programming languages, including Python, C++, Java, .NET, and MATLAB. In Python, Gurobi can be used through its Python API, allowing developers to build and solve optimization models in the Python environment. The Gurobi Python interface is very powerful, making it easy to leverage Python's data processing capabilities and other scientific computing libraries (such as NumPy and Pandas) to solve complex optimization problems.

## Google OR Tools

Google OR Tools is a powerful open-source software library for solving various optimization problems, including but not limited to vehicle routing problems, traveling salesman problems, and scheduling problems. Similar to Gurobi, Google OR Tools also supports multiple programming languages, such as Python, C++, and Java. In Python, OR Tools is easy to install and offers a wide range of functions to solve combinatorial optimization problems.

Overall, while both Gurobi and Google OR Tools can be used in Python, their applications are not limited to Python; they support cross-platform, cross-language optimization problem-solving. In Python, the library form of these tools makes it convenient to integrate them into Python-based data analysis, machine learning, or scientific computing projects.

## Application Example: Vehicle Routing Problem (VRP)

Suppose a company needs to deliver goods from a distribution center to multiple customers, with the goal of minimizing the total travel distance or cost. We can use Google OR Tools to solve this problem.

In [1]:
#pip install ortools

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.29.0 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.27.1 which is incompatible.


   ---------------------------------------- 0.0/130.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/130.2 MB 660.6 kB/s eta 0:03:18
   ---------------------------------------- 0.1/130.2 MB 1.3 MB/s eta 0:01:41
   ---------------------------------------- 0.1/130.2 MB 1.3 MB/s eta 0:01:41
   ---------------------------------------- 0.1/130.2 MB 1.3 MB/s eta 0:01:41
   ---------------------------------------- 0.1/130.2 MB 514.3 kB/s eta 0:04:13
   ---------------------------------------- 0.5/130.2 MB 2.0 MB/s eta 0:01:07
   ---------------------------------------- 0.6/130.2 MB 2.0 MB/s eta 0:01:04
   ---------------------------------------- 0.6/130.2 MB 2.0 MB/s eta 0:01:04
   ---------------------------------------- 0.7/130.2 MB 1.8 MB/s eta 0:01:14
   ---------------------------------------- 1.2/130.2 MB 2.7 MB/s eta 0:00:48
    --------------------------------------- 1.9/130.2 MB 3.8 MB/s eta 0:00:35
    --------------------------------------- 1.9/130.2 MB 3.8 MB/s e

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 10, 15, 20],  # Distance matrix
        [10, 0, 35, 25],
        [15, 35, 0, 30],
        [20, 25, 30, 0],
    ]
    data['num_vehicles'] = 2  # Number of vehicles
    data['depot'] = 0  # Starting point (depot)
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}miles\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

if __name__ == '__main__':
    main()

    

Objective: 80 miles
Route for vehicle 0:
 0 -> 0
Distance of the route: 0miles

Route for vehicle 1:
 0 -> 1 -> 3 -> 2 -> 0
Distance of the route: 80miles



This code utilizes Google OR Tools to solve the Vehicle Routing Problem (VRP), a classic optimization problem where a fleet of vehicles must visit a set of customers in the most efficient way possible. Here's an explanation and summary of the code:

#### 1 create_data_model(): This function initializes the problem data:

distance_matrix: Matrix representing the distances between locations (nodes). It's symmetric where distance_matrix[i][j] represents the distance from node i to node j.
num_vehicles: Number of vehicles available for routing.
depot: Starting point (depot) from which all vehicles start and end their routes.
#### 2 print_solution(): This function prints the solution obtained from the VRP solver:

It prints the objective value (total distance traveled by all vehicles).
It iterates over each vehicle's route, printing the sequence of nodes visited and the total distance traveled for each route.
#### 3 main(): The main function:

Instantiates the data problem.
Creates a RoutingIndexManager object to manage indices and provides lookup functions for nodes.
Creates a RoutingModel object.
Defines a distance callback function that returns the distance between two nodes based on the distance_matrix.
Registers the transit callback with the routing model.
Sets the cost of each arc (edge) in the routing model based on the distance callback.
Sets the first solution heuristic strategy to PATH_CHEAPEST_ARC.
Solves the VRP using the routing model and specified search parameters.
If a solution is found, it prints the solution using the print_solution() function.
#### 4 Summary:

The code efficiently solves the Vehicle Routing Problem using Google OR Tools.
It constructs the problem by defining the data, creating the routing model, setting up callbacks for distance computation, and specifying the search parameters.
Once solved, it prints the optimal solution, including the routes taken by each vehicle and the total distance traveled.

## optimization problem in a recommendation system.

In a recommendation system, the goal is to recommend the most suitable items to users based on their interests. The objective of this problem is to maximize the total interest score while satisfying constraints for each user and each item.

#### Data Model
Number of Users (num_users): There are 3 users in this example.

Number of Items (num_items): There are 4 items in this example.

Interest Scores Matrix (interest_scores): Each user has an interest score for each item, representing their level of preference for that item. These scores are organized into a matrix, where each row represents a user and each column represents an item.

Maximum Recommendations per Item (max_recommendations_per_item): The maximum number of times each item can be recommended.

Number of Recommendations per User (num_recommendations_per_user): The desired number of recommended items per user.

In [3]:
from ortools.linear_solver import pywraplp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # 用户数量
    data['num_users'] = 3
    # 物品数量
    data['num_items'] = 4
    # 用户对每个物品的兴趣得分矩阵
    data['interest_scores'] = [
        [8, 6, 5, 3],  # User 0's interest scores for items
        [1, 10, 7, 4],  # User 1's interest scores for items
        [2, 4, 9, 8],  # User 2's interest scores for items
    ]
    # 每个物品的推荐次数上限
    data['max_recommendations_per_item'] = 2
    # 每个用户推荐物品数量
    data['num_recommendations_per_user'] = 2
    return data

def main():
    """Solve the recommendation optimization problem."""
    data = create_data_model()

    # Create the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # Variables
    x = {}
    for u in range(data['num_users']):
        for i in range(data['num_items']):
            x[(u, i)] = solver.BoolVar(f'x_{u}_{i}')

    # Constraints
    # Each user gets exactly 'num_recommendations_per_user' recommendations
    for u in range(data['num_users']):
        solver.Add(sum(x[(u, i)] for i in range(data['num_items'])) == data['num_recommendations_per_user'])

    # Each item is recommended at most 'max_recommendations_per_item' times
    for i in range(data['num_items']):
        solver.Add(sum(x[(u, i)] for u in range(data['num_users'])) <= data['max_recommendations_per_item'])

    # Objective: maximize the total interest score
    objective = solver.Objective()
    for u in range(data['num_users']):
        for i in range(data['num_items']):
            objective.SetCoefficient(x[(u, i)], data['interest_scores'][u][i])
    objective.SetMaximization()

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found:')
        for u in range(data['num_users']):
            print(f'User {u} recommendations:')
            for i in range(data['num_items']):
                if x[(u, i)].solution_value() > 0.5:
                    print(f'  Item {i} with score {data["interest_scores"][u][i]}')
        print(f'Total interest score: {objective.Value()}')
    else:
        print('No solution found.')

if __name__ == '__main__':
    main()


Solution found:
User 0 recommendations:
  Item 0 with score 8
  Item 1 with score 6
User 1 recommendations:
  Item 1 with score 10
  Item 2 with score 7
User 2 recommendations:
  Item 2 with score 9
  Item 3 with score 8
Total interest score: 48.0


This code solves an optimization problem in a recommendation system using Google OR Tools. Here's an explanation of each part of the code:

#### 1 create_data_model(): This function creates a dictionary data containing the problem data:

num_users: Number of users.
num_items: Number of items.
interest_scores: Matrix of interest scores where each row represents a user and each column represents an item.
max_recommendations_per_item: Maximum number of times each item can be recommended.
num_recommendations_per_user: Desired number of recommended items per user.
#### 2 main(): This is the main function where the optimization problem is defined and solved.

A solver is created using SCIP.
Decision variables x[(u, i)] are created using BoolVar(), representing whether item i is recommended to user u.
Constraints are added:
Each user must receive exactly num_recommendations_per_user recommendations.
Each item can be recommended at most max_recommendations_per_item times.
The objective is to maximize the total interest score, so we define the objective function by setting coefficients based on interest scores.
The problem is solved using the solver.
If an optimal solution is found, the recommended items for each user and the total interest score are printed.
#### 3 Summary:

The code defines a recommendation system optimization problem where the goal is to maximize the total interest score by recommending items to users.
It formulates the problem as a linear programming problem and uses OR Tools to solve it.
Constraints ensure that each user receives the desired number of recommendations and that each item is recommended within the specified limit.
The objective function maximizes the total interest score based on users' interest scores for each item.
Finally, it prints the recommended items for each user and the total interest score if an optimal solution is found. If no solution is found, it prints "No solution found."